In [7]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import json
import shapely
import geopandas as gpd

In [8]:
df = pd.read_csv("../data/northeast - northeast.csv").drop(columns=['Unnamed: 0', 'state', 'address', 'company', 'plant_access', 'false negative'])
df.head(5)

,geometry,index,true positive,false positive
0,POINT (-77.93776233164682 35.994855621300836),16714,0,1
1,POINT (-77.92529932224166 35.99343682817532),16715,0,1
2,POINT (-77.94270133967557 35.99150292717033),16716,0,1
3,POINT (-77.98518552248538 36.00260360302083),16723,1,0
4,POINT (-77.98553984793107 36.00216825681871),16724,1,0


In [9]:
color_dictionary = dict({0: 'red',
                   1: 'green'})

label_dictionary = dict({'green': 'True Positive',
                         'red': 'False Positive'})

In [10]:
df['color'] = df['true positive'].map(color_dictionary)
df['label'] = df['color'].map(label_dictionary)
df.head(5)

,geometry,index,true positive,false positive,color,label
0,POINT (-77.93776233164682 35.994855621300836),16714,0,1,red,False Positive
1,POINT (-77.92529932224166 35.99343682817532),16715,0,1,red,False Positive
2,POINT (-77.94270133967557 35.99150292717033),16716,0,1,red,False Positive
3,POINT (-77.98518552248538 36.00260360302083),16723,1,0,green,True Positive
4,POINT (-77.98553984793107 36.00216825681871),16724,1,0,green,True Positive


In [11]:
df.color.value_counts()

color
green    228
red      108
Name: count, dtype: int64

In [12]:
NC_LAT = 36.0818
NC_LNG = -78.0575

In [13]:
map = folium.Map(location=[NC_LAT, NC_LNG], zoom_start=10)
geo_df = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=df["geometry"].apply(shapely.wkt.loads))

In [14]:
for index, row in geo_df.iterrows():
    folium.Marker(
        location = [row['geometry'].y, row['geometry'].x],
        icon = folium.Icon(color=row['color']),
        popup = row['label']
    ).add_to(map)

In [15]:
map

In [16]:
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map)

In [17]:
map

In [18]:
def add_points(state_map: folium.Map, state_df: pd.DataFrame, color: str):
    """Adds markers to a given state map based on location to represent farms.

    Args:
        state_map: map to add markers to
        state_df: DataFrame containing the farms and locations to be added to the map
        color: color for the markers as a string

    Returns:
        N/A, adds points to existing map

    """
    for index, location_info in state_df.iterrows():
        folium.Marker([location_info["lat"], location_info["long"]],\
            popup=location_info["name"], icon=folium.Icon(color=color)).add_to(state_map)

In [20]:
cafos = pd.read_csv("../data/clean/cleaned_cafo_poultry.csv")
nc_cafos = cafos[cafos['state']=="NC"]

In [21]:
add_points(map, nc_cafos, "blue")

In [22]:
map

In [82]:
map.save('../html/color_coded_northeast.html')